In [1]:
import os
import sys
root = '\\'.join(os.getcwd().split('\\')[:-1])
sys.path.append(root)
from modules.prompt_config.prompt_config import PromptConfig
from modules.context_maker.context_maker import ContextMaker
from modules.databases_sql.databases import UserDatabase
from langchain.chat_models import ChatOpenAI
from modules.global_configs import OPENAI_API_KEY
from IPython.display import display, Markdown

PROMPTS_PATH = os.path.join(root , "documents" , "master_prompts")
SUMMARY_CONVERSATION_PROMPT_PATH = os.path.join(PROMPTS_PATH , "summary_conversations" , "summary_conversation_prompt_v0_english.txt")
SUMMARY_CONVERSATION = {'path': SUMMARY_CONVERSATION_PROMPT_PATH, 'input_variables': ['n_messages_before', 'conversation']}

CONTEXT_MAKER_PROMPT_PATH = os.path.join(PROMPTS_PATH , "context_maker" , "context_maker_prompt.txt")
CONTEXT_MAKER = {'path': CONTEXT_MAKER_PROMPT_PATH, 'input_variables': ['conversations_summaries', 'emotions']}

cambalache_model = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0.1, api_key=OPENAI_API_KEY)

In [2]:
cambalache_model = ChatOpenAI(model_name = 'gpt-4o', temperature = 0.1, api_key=OPENAI_API_KEY)

In [3]:
prompt_manager = PromptConfig()
prompt_manager.load_prompt(SUMMARY_CONVERSATION['path'], "conversation_summaries",SUMMARY_CONVERSATION['input_variables'])
prompt_manager.load_prompt(CONTEXT_MAKER['path'], "context_maker", CONTEXT_MAKER['input_variables'])

In [4]:
context_maker = ContextMaker()
context_maker.load_prompt(prompt_manager.get_prompt("context_maker"), model=cambalache_model)

In [5]:
userdb = UserDatabase(path=os.path.join(root, "users", "farzad", "farzad.db"))

In [6]:
def conversation_summaries(self, format='str', session_id=None):
    """
    format = 'str' or 'list'
    """
    sessions = self.get_session_summary_data_as_dataframe(session_id=session_id)

    if len(sessions) == 0:
        return ''
    
    if format == 'str':
        str_ = ''
        for i in range(len(sessions)):
            str_ += f"Conversation {i+1}:[{sessions['summary'][i]}]\n"
        return str_
    elif format == 'list':
        return sessions['summary'].to_list()


In [7]:
conversation_summaries_ = conversation_summaries(userdb, format='str')

In [8]:
dashboard = str(userdb.get_user_dashboard())

In [9]:
context_maker.predict(conversation_summaries=conversation_summaries_, emotions=dashboard)

In [10]:
display(Markdown(context_maker.get_information()))

**Key themes and recurring topics discussed:**
- Feeling unappreciated and overwhelmed at work.
- Dealing with personal issues affecting focus.
- Feeling undermined and not being listened to by colleagues and HR manager.

**Feelings expressed by the patient and emotional patterns:**
- Feeling down, unhappy, overwhelmed, frustrated, dissatisfied, and undermined.
- Emotional indicators show high levels of disappointment, annoyance, and sadness.
- Stress and depression scores have increased significantly.

**Information provided by the patient:**
- Feeling unappreciated at work and overworked.
- Not getting enough pay.
- Dealing with personal issues like a potential divorce.
- Trying to communicate with colleagues and HR manager but not being listened to.

**Requests and significant events or issues mentioned by the patient:**
- Seeking understanding and support.
- Discussing personal and work-related issues.
- Seeking support and understanding from colleagues and HR manager.

**Assistant's behavior:**
- Showing empathy and actively listening to the patient's concerns.
- Asking for more details to understand the patient's situation better.

**Recommendations provided by the assistant:**
- None mentioned.

**Goals provided by the assistant:**
- None mentioned.

**Have there been any incidents (YES/NO):**
- NO

**Have there been any greetings (YES/NO):**
- YES

**Suicidal thoughts (YES/NO):**
- NO